In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import os
import sys
import L96sim

from L96_emulator.util import dtype, dtype_np, device

res_dir = '/gpfs/work/nonnenma/results/emulators/L96/'
data_dir = '/gpfs/work/nonnenma/data/emulators/L96/'

In [ ]:
from L96_emulator.eval import get_rollout_fun, plot_rollout
from L96_emulator.parametrization import Parametrized_twoLevel_L96, Parametrization_lin, Parametrization_nn
from L96_emulator.networks import Model_forwarder_rk4default
from L96_emulator.run_parametrization import setup_parametrization
from L96_emulator.data_assimilation import get_model
from L96_emulator.util import as_tensor, dtype_np, sortL96intoChannels, sortL96fromChannels
from L96sim.L96_base import f1, f2, pf2
import numpy as np
import torch
import os

def get_eval_parametrization(exp_ids, 
                             data=None, 
                             n_start_rollout=10, T_dur=5000, 
                             T_data=1000, N_trials=1000,
                             model_exp_id = None, 
                             ifplot=False):

    T_start = None 
    i_trial = None     

    for exp_id in exp_ids:
        exp_names = os.listdir('experiments_parametrization/')   
        conf_exp = exp_names[np.where(np.array([name[:2] for name in exp_names])==str(exp_id))[0][0]][:-4]
        args = setup_parametrization(conf_exp=f'experiments_parametrization/{conf_exp}.yml')
        args.pop('conf_exp')

        save_dir = 'results/parametrization/' + args['exp_id'] + '/'
        out = np.load(res_dir + save_dir + 'out.npy', allow_pickle=True)[()]

        X_init = out['X_init']

        model_pars = {
            'exp_id' :  args['model_exp_id'] if model_exp_id is None else model_exp_id, # 44: analytical bilinNet
            'model_forwarder' : args['model_forwarder'],
            'K_net' : args['K'],
            'J_net' : 0,
            'dt_net' : args['dt']
        }

        ##########################
        #       models           #
        ##########################

        # trained parametrized model
        model, model_forwarder, _ = get_model(model_pars, res_dir=res_dir, exp_dir='')

        if args['parametrization'] == 'linear':
            param_train = Parametrization_lin(a=as_tensor(out['param_train_state_dict']['a']), 
                                              b=as_tensor(out['param_train_state_dict']['b']))
            param_offline = Parametrization_lin(a=as_tensor(out['param_offline_state_dict']['a']), 
                                              b=as_tensor(out['param_offline_state_dict']['b']))
        elif args['parametrization'] == 'nn':
            param_train = Parametrization_nn(n_hiddens=args['n_hiddens'], 
                                             kernel_size=args['kernel_size'],
                                             n_in=1,n_out=1)
            for key, value in out['param_train_state_dict'].items():
                out['param_train_state_dict'][key] = as_tensor(value)
            param_train.load_state_dict(out['param_train_state_dict'])
            param_offline = Parametrization_nn(n_hiddens=args['n_hiddens'], 
                                               kernel_size=args['kernel_size'],
                                               n_in=1,n_out=1)
            for key, value in out['param_offline_state_dict'].items():
                out['param_offline_state_dict'][key] = as_tensor(value)
            param_offline.load_state_dict(out['param_offline_state_dict'])
        model_parametrized_train = Parametrized_twoLevel_L96(emulator=model, parametrization=param_train)
        model_forwarder_parametrized_train = Model_forwarder_rk4default(model=model_parametrized_train, 
                                                                        dt=args['dt'])
        model_parametrized_offline = Parametrized_twoLevel_L96(emulator=model, parametrization=param_offline)
        model_forwarder_parametrized_offline = Model_forwarder_rk4default(model=model_parametrized_offline, 
                                                                        dt=args['dt'])

        # initial and reference parametrized models
        param_ref = Parametrization_lin(a=as_tensor(np.array([-0.31])), b=as_tensor(np.array([-0.2])))
        param_init = Parametrization_lin(a=as_tensor(np.array([-0.75])), b=as_tensor(np.array([-0.4])))
        model_parametrized_init = Parametrized_twoLevel_L96(emulator=model, parametrization=param_init)
        model_forwarder_parametrized_init = Model_forwarder_rk4default(model=model_parametrized_init, 
                                                                       dt=args['dt'])
        model_parametrized_ref = Parametrized_twoLevel_L96(emulator=model, parametrization=param_ref)
        model_forwarder_parametrized_ref = Model_forwarder_rk4default(model=model_parametrized_ref, 
                                                                      dt=args['dt'])

        # ground-truth high-res model
        dX_dt_oneLevel = np.empty(args['K'], dtype=dtype_np)
        dX_dt_twoLevel = np.empty((args['K']*(args['J']+1),n_start_rollout), dtype=dtype_np)
        def fun_oneLevel(t, x):
            return f1(x, args['l96_F'], dX_dt_oneLevel, args['K']).copy()
        def fun_twoLevel(t, x):
            return pf2(x, args['l96_F'], args['l96_h'], args['l96_b'], args['l96_c'], 
                      dX_dt_twoLevel, args['K'], args['J']).copy().T
        
        class Torch_solver(torch.nn.Module):
            # numerical solver (from numpy/numba/Julia)
            def __init__(self, fun):
                self.fun = fun
            def forward(self, x):
                J = x.shape[-2] - 1
                if J == 0:
                    x = sortL96fromChannels(x.detach().cpu().numpy()).flatten()
                else:
                    x = sortL96fromChannels(x.detach().cpu().numpy()).T
                return as_tensor(sortL96intoChannels(np.atleast_2d(self.fun(0., x)), J=J))


        model_forwarder_np_twoLevel = Model_forwarder_rk4default(Torch_solver(fun_twoLevel), dt=args['dt'])
        model_forwarder_np_oneLevel = Model_forwarder_rk4default(Torch_solver(fun_oneLevel), dt=args['dt'])

        ##########################
        #       test data        #
        ##########################

        from L96_emulator.util import rk4_default, get_data

        spin_up_time = 5.
        T = T_data*args['dt'] + spin_up_time

        try:
            print('data.shape', data.shape)
            assert data.shape == (N_trials, int(T/args['dt'])+1, args['K']*(args['J']+1))
            print('found data with matching specs (shape)')
        except:
            print('generating test data')
            data, _ = get_data(K=args['K'], J=args['J'], T=T, dt=args['dt'], N_trials=N_trials, 
                              F=args['l96_F'], h=args['l96_h'], b=args['l96_b'], c=args['l96_c'], 
                              resimulate=True, solver=rk4_default,
                              save_sim=False, data_dir=data_dir)

        ##########################
        #       rollouts         #
        ##########################

        model_forwarders = [model_forwarder_np_oneLevel,
                            model_forwarder_parametrized_init, 
                            model_forwarder_parametrized_train,
                            model_forwarder_parametrized_offline,
                            model_forwarder_parametrized_ref,
                            model_forwarder_np_twoLevel]


        Js = [0, 0, 0, 0, 0, args['J']]
        panel_titles=['no param.', 
                      'initial param.', 
                      'online param.', 
                      'offline param.', 
                      'reference param.', 
                      'two-level L96']
        n_start = n_start_rollout
        if T_start is None:
            T_start = np.linspace(int(spin_up_time/args['dt']), int(T/args['dt']), n_start).astype(np.int)
            i_trial = np.random.choice(N_trials, size=T_start.shape, replace=False)
        #print('T_start, i_tria', (T_start, i_trial))

        sols = np.nan * np.ones((len(model_forwarders), n_start, T_dur+1, args['K']))
        
        for i_model in range(len(model_forwarders[:-1])): 

            model_forwarder_i, J_i = model_forwarders[i_model], Js[i_model]

            def model_simulate(y0, dy0, n_steps):
                x = np.empty((n_steps+1, *y0.shape[1:]))
                x[0] = y0.copy()
                xx = as_tensor(x[0])
                for i in range(1,n_steps+1):
                    xx = model_forwarder_i(xx.reshape(-1,J_i+1,args['K']))
                    x[i] = xx.detach().cpu().numpy().copy()
                return x
            
            print('forwarding model ' + panel_titles[i_model])
            X_init = []
            for i in range(n_start):
                X_init.append(data[i_trial[i], T_start[i]] if N_trials > 1 else data[T_start[i]])
                X_init[-1] = sortL96intoChannels(X_init[-1][:args['K']*(J_i+1)],J=J_i)
            X_init = np.stack(X_init)
            X_init = X_init.reshape(1, *X_init.shape)
            with torch.no_grad():
                sol = model_simulate(y0=X_init, dy0=None, n_steps=T_dur)
            sols[i_model,:,:,:] = sol[:,:,0,:].transpose(1,0,2)

        # not parallelising Numba model over initial values for rollouts
        def model_simulate(y0, dy0, n_steps):
            x = np.empty((n_steps+1, *y0.shape[1:]))
            x[0] = y0.copy()
            xx = as_tensor(x[0]).reshape(1,1,-1)
            for i in range(1,n_steps+1):
                xx = model_forwarder_np_twoLevel(xx.reshape(-1,args['J']+1,args['K']))
                x[i] = xx.detach().cpu().numpy().copy()
            return x

        print('forwarding model ' + panel_titles[-1])
        X_init = []
        for i in range(n_start):
            X_init.append(data[i_trial[i], T_start[i]] if N_trials > 1 else data[T_start[i]])
            X_init[-1] = sortL96intoChannels(X_init[-1],J=args['J'])
        X_init = np.stack(X_init)
        X_init = X_init.reshape(1, *X_init.shape)
        with torch.no_grad():
            sol = model_simulate(y0=X_init, dy0=None, n_steps=T_dur)
        sols[-1,:,:,:] = sol[:,:,0,:].transpose(1,0,2)

        ##########################
        #  one-step predictions  #
        ##########################


        model_forwarders_eval = [model_forwarder_np_oneLevel,
                                 model_forwarder_parametrized_init, 
                                 model_forwarder_parametrized_train,
                                 model_forwarder_parametrized_offline,
                                 model_forwarder_parametrized_ref]


        MFWDs = [Model_forwarder_rk4default]
        dts = {Model_forwarder_rk4default : args['dt']}
        RMSEs_states = np.zeros((len(model_forwarders_eval), len(T_start)))

        print('\n')
        print('calculating RMSEs on system states')
        print('\n')
        
        
        inputs_torch = []
        for i in range(n_start):
            inputs = data[i_trial[i], T_start[i]] if N_trials > 1 else data[T_start[i]]
            inputs_torch.append(as_tensor(sortL96intoChannels(inputs,J=args['J'])))
        inputs_torch = torch.stack(inputs_torch)
        out_np = model_forwarder_np_twoLevel(inputs_torch)[:,:1,:]
        
        for m_i, model_forwarder in enumerate(model_forwarders_eval):

            # currently not parallelizing over one-level L96 (simulator parallelizable, but different input shape)
            if model_forwarder == model_forwarder_np_oneLevel:
                out_simulator =[]
                for i in range(len(T_start)):
                    inputs = data[i_trial[i], T_start[i]] if N_trials > 1 else data[T_start[i]]
                    inputs_torch = as_tensor(sortL96intoChannels(np.atleast_2d(inputs.copy()),J=args['J']))
                    out_model = model_forwarder(inputs_torch[:,:1,:])
                    RMSEs_states[m_i,i] = np.sqrt(((out_np[i] - out_model)**2).mean().detach().cpu().numpy())
            else: 
                inputs_torch = []
                for i in range(n_start):
                    inputs = data[i_trial[i], T_start[i]] if N_trials > 1 else data[T_start[i]]
                    inputs_torch.append(as_tensor(sortL96intoChannels(inputs,J=args['J'])))
                inputs_torch = torch.stack(inputs_torch)

                out_model = model_forwarder(inputs_torch[:,:1,:])
                RMSEs_states[m_i] = np.sqrt(((out_np - out_model)**2).mean(axis=(1,2)).detach().cpu().numpy())


        ##########################
        #         plot           #
        ##########################

        clrs = ['k', 'red', 'orange', 'green', 'purple', 'blue']
        if ifplot:
            fontsize=18
            plt.figure(figsize=(16,11))

            if args['parametrization'] == 'linear':
                model_forwarders_plot = [0,1,2,len(model_forwarders)-1]
            else:
                model_forwarders_plot = [0,2,len(model_forwarders)-1]

            for i,i_model in enumerate(model_forwarders_plot): 

                plt.subplot(2,len(model_forwarders_plot),len(model_forwarders_plot)-i) #plot right to left
                plt.imshow(sols[i_model,0].T, aspect='auto')
                plt.colorbar()
                plt.title(panel_titles[i_model], fontsize=fontsize)

                if i == len(model_forwarders_plot)-1:
                    plt.ylabel('location k', fontsize=fontsize)
                plt.xlabel('time [au]', fontsize=fontsize)

                plt.yticks([], fontsize=fontsize)
                plt.xticks([0, T_dur/2, T_dur], [0, args['dt']*T_dur/2, args['dt']*T_dur], fontsize=fontsize)


            ax = plt.subplot(2,3,6)
            xx = np.linspace(-7.5, 15, 20)
            parametrizations, labels = [param_offline, param_train ], ['offline', 'online']
            clrs_plot = ['g', 'orange', 'r', 'purple', 'k']
            if args['parametrization'] == 'linear':
                parametrizations += [param_ref, param_init]
                labels += ['ref.', 'init.']
            spinup = int(args['spin_up_time']/args['dt'])
            idx = np.linspace(0, 
                              int(args['T']/args['dt'])-1, 
                              np.min((25, int(args['T']/args['dt'])))).astype(np.int)
            X_plot, y_plot = out['X'][spinup:][idx], out['y'][spinup:][idx]

            plt.plot(X_plot.flatten(), y_plot.flatten(), 'b.')
            for i, parametrization in enumerate(parametrizations):
                plt.plot(xx, 
                         parametrization(as_tensor(xx.reshape(1,1,-1))).detach().cpu().numpy().flatten(),
                         color=clrs_plot[i],
                         linewidth=2.5,
                         label=labels[i])
                plt.xlabel(r'$x_k$', fontsize=fontsize)
                plt.ylabel(r'$\mathcal{B}(x_k)$', fontsize=fontsize)
                plt.legend(fontsize=fontsize)
                plt.axis([-7.5, 12.5, -6, 5])
                plt.xticks([-5, 0, 5, 10], fontsize=fontsize)
                plt.yticks([-6, -4, -2, 0, 2, 4], fontsize=fontsize)
                plt.grid(True)
            box = ax.get_position()
            box.y1 -= 0.1 * (box.y1-box.y0)
            ax.set_position(box)
                
            ax = plt.subplot(2,3,5)
            from scipy import signal
            print('computing PSD')
            for i_model in [0, 2, 3, len(sols)-1]:
                psds = []
                sig = sols[i_model]
                for loc in range(args['K']):
                    for i in range(len(sig)):
                        freqs, psd = signal.welch(sig[i,:,loc])
                        psds.append(psd)
                linestyle = ':' if panel_titles[i_model] == 'offline param.' else '-'
                plt.loglog(freqs/args['dt'], 
                           np.stack(psds).mean(axis=0), 
                           color = clrs[i_model],
                           linestyle=linestyle,
                           linewidth=2.5,
                           label=panel_titles[i_model])
            plt.xlabel('Frequency', fontsize=fontsize)
            plt.ylabel('Power spectral density', fontsize=fontsize)
            plt.xticks(fontsize=fontsize)
            plt.yticks([1e2, 1e-2, 1e-6, 1e-10], fontsize=fontsize)
            ax.spines['top'].set_visible(False)
            ax.spines['right'].set_visible(False)
            plt.legend(fontsize=fontsize, frameon=False, loc=(0.05,0.05), handlelength=0.7)
            box = ax.get_position()
            box.x0 += 0.1 * (box.x1-box.x0)
            box.x1 -= 0.05 * (box.x1-box.x0)
            box.y1 -= 0.1 * (box.y1-box.y0)
            ax.set_position(box)
            """
            model_labels = ['one-level', 
                          'init.', 
                          'trained', 
                          'offline',
                          'ref.']
            
            RMSEs = RMSEs_states
            if args['parametrization'] == 'linear':
                model_rmses = range(len(model_labels))
            else:
                model_rmses = [0, 2, 3, 4]
            for i, i_model in enumerate(model_rmses):
                if args['parametrization'] == 'nn' and model_labels[i_model] == 'ref.':
                    model_labels[i_model] = 'linear'
                plt.semilogy(i*np.ones(2)+np.array([-0.5,0.5]), 
                         RMSEs.mean(axis=1)[i_model]*np.ones(2),
                         color=clrs[i_model], linewidth=1.5)
                plt.semilogy(i*np.ones(2), 
                         RMSEs.mean(axis=1)[i_model]+RMSEs.std(axis=1)[i_model]*np.array([-1,1]),
                         color='k', linewidth=1.5)
            plt.yticks(fontsize=fontsize)
            plt.xticks(np.arange(len(model_rmses)), [model_labels[j] for j in model_rmses], fontsize=fontsize)
            plt.ylabel('RMSE', fontsize=fontsize)
            ax.spines['top'].set_visible(False)
            ax.spines['right'].set_visible(False)
            """
            rmses = np.zeros((len(model_forwarders), n_start, T_dur+1))
            ax = plt.subplot(2,3,4)
            plot_models = [0, 2, 3, 4]
            if args['parametrization'] == 'linear':
                plot_models += [1]
            for i_model in plot_models:
                if args['parametrization'] == 'nn' and panel_titles[i_model] == 'reference param.':
                    panel_titles[i_model] = 'linear param.'
                
                for i in range(n_start):
                    rmses[i_model,i,:] = np.sqrt(np.mean((sols[i_model,i] - sols[-1,i])**2, axis=1))
                linestyle = ':' if panel_titles[i_model] == 'offline param.' else '-'
                plt.semilogy(rmses[i_model].mean(axis=0), 
                             linestyle=linestyle, 
                             color=clrs[i_model], 
                             linewidth=2.5,
                             label=panel_titles[i_model])
                    
            plt.ylabel('RMSE', fontsize=fontsize)
            plt.xlabel('time [au]', fontsize=fontsize)
            plt.xticks([0, T_dur/2, T_dur], [0, args['dt']*T_dur/2, args['dt']*T_dur], fontsize=fontsize)
            plt.yticks(fontsize=fontsize)
            ax.spines['top'].set_visible(False)
            ax.spines['right'].set_visible(False)
            plt.axis([-1, T_dur+1, 5e-2, np.max(rmses.mean(axis=1))])
            box = ax.get_position()
            box.x0 += 0.05 * (box.x1-box.x0)
            box.x1 -= 0.15 * (box.x1-box.x0)
            box.y1 -= 0.1 * (box.y1-box.y0)
            ax.set_position(box)            
            plt.legend(fontsize=fontsize, frameon=False, handlelength=0.7, loc=(0.35, 0.05))

            plt.gcf().text(0.105,   0.88, 'a)', fontsize=fontsize, weight='bold')
            plt.gcf().text(0.105,   0.45, 'b)', fontsize=fontsize, weight='bold')
            plt.gcf().text(0.36,    0.45, 'c)', fontsize=fontsize, weight='bold')            
            plt.gcf().text(0.62,    0.45, 'd)', fontsize=fontsize, weight='bold')            
            
            #plt.savefig(res_dir + 'figs/param.pdf', bbox_inches='tight', pad_inches=0, frameon=False)
            plt.show()

    return data

In [ ]:
try: 
    data.shape
except: 
    data = None

In [ ]:
exp_ids = ['20']
data = get_eval_parametrization(exp_ids=exp_ids, 
                                T_data=1,
                                N_trials=1000,
                                n_start_rollout=1000,
                                T_dur=500,
                                data=data, 
                                model_exp_id = '44',                                
                                ifplot=True)

In [ ]:
exp_ids = ['22']
data = get_eval_parametrization(exp_ids=exp_ids, 
                                T_data=1,
                                N_trials=1000,
                                n_start_rollout=1000,
                                T_dur=500,
                                data=data, 
                                model_exp_id = '44',                                
                                ifplot=True)

In [ ]:
exp_ids = ['23']
data = get_eval_parametrization(exp_ids=exp_ids, 
                                T_data=1,
                                N_trials=1000,
                                n_start_rollout=1000,
                                T_dur=5000,
                                data=data, 
                                model_exp_id = '44',                                
                                ifplot=True)

In [ ]:
exp_ids = ['24']
data = get_eval_parametrization(exp_ids=exp_ids, 
                                T_data=1,
                                N_trials=1000,
                                n_start_rollout=1000,
                                T_dur=5000,
                                data=data, 
                                model_exp_id = '44',                                
                                ifplot=True)

In [ ]:
exp_ids = ['25']
data = get_eval_parametrization(exp_ids=exp_ids, 
                                T_data=1,
                                N_trials=1000,
                                n_start_rollout=1000,
                                T_dur=5000,
                                data=data, 
                                model_exp_id = '44',                                
                                ifplot=True)

In [ ]:
exp_ids = ['26']
data = get_eval_parametrization(exp_ids=exp_ids, 
                                T_data=1,
                                N_trials=1000,
                                n_start_rollout=1000,
                                T_dur=5000,
                                data=data, 
                                model_exp_id = '44',                                
                                ifplot=True)

# directly plot learned parametrization functions

In [ ]:
from L96_emulator.eval import get_rollout_fun, plot_rollout
from L96_emulator.parametrization import Parametrized_twoLevel_L96, Parametrization_lin, Parametrization_nn
from L96_emulator.networks import Model_forwarder_rk4default
from L96_emulator.run_parametrization import setup_parametrization
from L96_emulator.data_assimilation import get_model
from L96_emulator.util import as_tensor, dtype_np, sortL96intoChannels, sortL96fromChannels
from L96sim.L96_base import f1, f2, pf2
import numpy as np
import torch
import os


fontsize = 14

def plot_parametrization(exp_ids):


    xx = np.linspace(-7.5, 15, 20)
    plt.figure(figsize=(16,8))

    for exp_id in exp_ids:
        exp_names = os.listdir('experiments_parametrization/')   
        conf_exp = exp_names[np.where(np.array([name[:2] for name in exp_names])==str(exp_id))[0][0]][:-4]
        args = setup_parametrization(conf_exp=f'experiments_parametrization/{conf_exp}.yml')
        args.pop('conf_exp')

        save_dir = 'results/parametrization/' + args['exp_id'] + '/'
        out = np.load(res_dir + save_dir + 'out.npy', allow_pickle=True)[()]

        X_init = out['X_init']

        model_pars = {
            'exp_id' : args['model_exp_id'],
            'model_forwarder' : args['model_forwarder'],
            'K_net' : args['K'],
            'J_net' : 0,
            'dt_net' : args['dt']
        }

        ##########################
        #       models           #
        ##########################

        # trained parametrized model
        model, model_forwarder, _ = get_model(model_pars, res_dir=res_dir, exp_dir='')

        if args['parametrization'] == 'linear':
            param_train = Parametrization_lin(a=as_tensor(out['param_train_state_dict']['a']), 
                                              b=as_tensor(out['param_train_state_dict']['b']))
        elif args['parametrization'] == 'nn':
            param_train = Parametrization_nn(n_hiddens=args['n_hiddens'], n_in=1,n_out=1)
            for key, value in out['param_train_state_dict'].items():
                out['param_train_state_dict'][key] = as_tensor(value)
            param_train.load_state_dict(out['param_train_state_dict'])


        ##########################
        #         plot           #
        ##########################
        
        ln = '--' if len(args['offset']) > 1 else '-'
        mrkr = 'd' if len(args['offset']) > 1 else 'x'
        if args['T'] == 10:
            clr = 'orange'
        elif args['T'] == 1:
            clr = 'blue'
        elif args['T'] == 0.2:
            clr = 'black'
            

        label = args['parametrization'] + '_N' + str(int(1000*args['T']*args['train_frac'])) + '_n' + str(len(args['offset']))
        
        if args['parametrization'] == 'linear' :
            plt.subplot(1,2,1)
        else:
            plt.subplot(1,2,2)
        plt.plot(xx, 
                 param_train(as_tensor(xx.reshape(1,1,-1))).detach().cpu().numpy().flatten(),
                 ln,
                 marker=mrkr,
                 color=clr,
                 linewidth=2.5,
                 label=label)
    for i in range(2):
        plt.subplot(1,2,i+1)    
        plt.xlabel(r'$x_k$', fontsize=fontsize)
        plt.ylabel(r'$\mathcal{B}(x_k)$', fontsize=fontsize)
        plt.legend(fontsize=fontsize)
        plt.axis([-7.5, 12.5, -6, 5])
        plt.xticks([-5, 0, 5, 10])
        plt.yticks([-6, -4, -2, 0, 2, 4])
        plt.grid(True)
    plt.show()


plot_parametrization(exp_ids=np.array(['01','02','03','04','05','06','07','08','09','10','11','12']))
